<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Data Analysis</center></h1>
<h2><center>Seminar: Ensemble Learning. </center></h2>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

## Task 1

Load data from `data/x_data.tsv` and `data/y_data.tsv`. 

These are descriptive features and target feature. Put them in `X` and `y` respectively.

## Task 2

Plot validation curve w.r.t. hyperparameter `max_depth` of Decistion Tree? Which depth deliveres the best roc-auc?

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier


## Task 3

Wrap DecTree in to BaggingClasifier. Repeat the same procedure with `max_depth`. 

What are you observing? Is quality better?

HINT: to iterate over base_estimator's hyperparameters use `"base_estimator_"` prefix in `param_name` argument of `validation curve`

In [ ]:
from sklearn.ensemble import BaggingClassifier

## Task 4

Now consider RandomForestClassifier. Repeat the same actions with it

In [ ]:
from sklearn.ensemble import RandomForestClassifier

## Task 5

Tree leaves can be considered as categorical features:

* Train forest with $T$ trees on your data
* Enumerate leaves of each tree from $0$ to $L_T$
* Each object would have feature "a which leaf it ended up" for each tree
* Encode those with OHE
* Profit??

Any doubts?

In [ ]:
from sklearn.ensemble import RandomTreesEmbedding

In [ ]:
emb = RandomTreesEmbedding(n_estimators=10, max_depth=5, random_state=123)
emb.fit(X)

# emb = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=123)
# emb.fit(X, y)

In [ ]:
X_emb = emb.apply(X)

In [ ]:
X.shape

In [ ]:
X_emb.shape

In [ ]:
X_emb[:10]

In [ ]:
X_emb_sparse = emb.transform(X)

In [ ]:
X_emb_sparse

In [ ]:
model = Pipeline([
    ('tree_emb', RandomTreesEmbedding(n_estimators=50, max_depth=4)),
    ('clf', LogisticRegression(C=0.1))
])

simple_model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(C=0.1))
])


In [ ]:
scores = cross_val_score(model, X, y, scoring=scoring, cv=cv, n_jobs=-1)
scores.mean()

In [ ]:
scores = cross_val_score(simple_model, X, y, scoring=scoring, cv=cv, n_jobs=1)
scores.mean()

# Task 6 (staking)

In [ ]:
!pip install vecstack

In [ ]:
from vecstack import stacking

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier

In [ ]:
models = [
    RandomForestClassifier(n_estimators=50, max_depth=5, random_state=123),
    RandomForestClassifier(n_estimators=50, max_depth=10, random_state=123, max_features='log2'),
    Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression())
    ]),
    Pipeline([
        ('scaler', StandardScaler()),
        ('clf', KNeighborsClassifier(n_neighbors=11))
    ]),
    GaussianNB(),
    DummyClassifier(random_state=123),
    DummyClassifier(random_state=321)
]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=123, test_size=0.2)

In [ ]:
def roc_auc_score_cust(y_true, y_hat):
    return roc_auc_score(y_true, y_hat[:,1])

In [ ]:
S_train, S_valid = stacking(models,
                               X_train, y_train, X_valid,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=123,         
                               verbose=2)                

In [ ]:
S_train.shape

In [ ]:
S_valid.shape

In [ ]:
last_model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=0.01, 
                                       random_state=123)

In [ ]:
last_model.fit(S_train, y_train)
y_hat = last_model.predict_proba(S_valid)
roc_auc_score(y_valid, y_hat[:, 1])

In [ ]:
last_model.feature_importances_

In [ ]:
for model in models:
    last_model.fit(X_train, y_train)
    y_hat = model.predict_proba(X_valid)
    score = roc_auc_score(y_valid, y_hat[:, 1])
    print('{}: {}'.format(model.__str__().split('(')[0], score))